<div style="border:solid blue 2px; padding: 20px">
  
**Привет, Вадим!**

Меня зовут Юлия Никишина, я ревьюер, буду проверять твой проект. 

Если не против, предлагаю общаться на "ты". Если это неприемлемо, обязательно напиши мне в комментариях, и мы перейдем на "Вы".

Далее в файле ты увидишь мои комментарии в цветных ячейках с пометкой "Комментарий ревьюера". Цвет ячеек зависит от того, какие действия от тебя требуются, ниже указано подробнее. 

**Важно:** Пожалуйста, не удаляй и не изменяй мои комментарии. При повторной проверке я буду ориентироваться на них. 

<div class="alert alert-success"; style="border-left: 7px solid green">
<b>✅ Комментарий ревьюера (хорошо)</b> 
    
Такими комментариями я буду отмечать хорошо проделаную работу.
</div>

<div class="alert alert-warning"; style="border-left: 7px solid gold">
<b>⚠️ Комментарий ревьюера (обрати внимание)</b> 
    
Части проекта, помеченные такими комментариями, можно не исправлять, но стоит обратить на них внимание. В таких комментариях я постараюсь предложить альтернативный вариант решения или дам рекомендации на будущее. 
</div>

<div class="alert alert-danger"; style="border-left: 7px solid red">
<b>⛔️ Комментарий ревьюера (нужно исправить)</b> 
    
Если ты видишь такой комментарий, значит я указываю тебе на недочёт, его нужно исправить. Без такого исправления я не смогу принять твою работу.
</div>

Ты можешь под моими комментариями оставлять свои, описывать в них сделанные исправления или задавать вопросы, выделяя их другим цветом. 

<div class="alert alert-info"; style="border-left: 7px solid blue">
<b>Комментарий студента</b>

<div style="border:solid blue 2px; padding: 20px">
  
**Заключительный комментарий, ревью 3**
    
Вадим, привет. 
    
Спасибо за корректировку. Запрос теперь составлен верно. Твой проект принят, поздравляю. 

---

Как ты возможно уже знаешь, SQL - важный скилл для аналитика данных, его знание требуется требуется в большом количестве вакансий, поэтому стоит его регулярно практиковать, чтобы не забывался. Если вдруг тебе не хватает повседневной практики, могу порекомендовать следующие ресурсы:
    
1. [Бесплатный курс на Stepik "Введение в базы данных"](https://stepik.org/course/551/syllabus)
2. [Бесплатный тренажер по SQL там же](https://stepik.org/course/63054)
3. [Еще недорогой краткий курс с той же платформы об оконных функциях](https://stepik.org/course/95367/syllabus)
4. Каналы в телеграмме, в которых можно задать вопросы по SQL
    - [sqlcom](https://t.me/sqlcom)
    - [sql_ninja](https://t.me/sql_ninja)
5. Еще несколько тренажеров, платных, но с бесплатной частью
   - [Тренажер SQLhabit](https://www.sqlhabit.com/)
   - [Тренажер SQLzoo](https://sqlzoo.net/wiki/SQL_Tutorial)
   - [Тренажер LearnDB](https://learndb.ru/courses)
   - [Тренажер W3Schools](https://www.w3schools.com/sql/)
    
6. Из моих последних открытий парочка тренажеров, в которых задачи реально похожи на бизнесовые задачи. Но второй на английском:
    
    - [SQL Academy](https://sql-academy.org/ru/trainer)
    - [StrataScratch](https://platform.stratascratch.com/coding?code_type=1)
    
Поздравляю тебя с завершением финального спринта!

<div style="border:solid blue 2px; padding: 20px">
  
**Заключительный комментарий, ревью 2**
    
Вадим, привет. Спасибо за доработки, особенно за то, что принял во внимание не только критичные замечания, но и рекомендации. 
    
Остается вопрос с запросом для определения автора в самой высокой средней оценкой книг. Я попыталась написать подробнее, что необходимо было сделать с предыдущей версией запроса. Можешь, пожалуйста, доработать запрос?
    
Буду ждать обновленный проект.

<div style="border:solid blue 2px; padding: 20px">
  
**Заключительный комментарий**
    
Вадим, спасибо за проект. У тебя получилась хорошая работа. 
    
Ты придерживался последовательности, указанной в задании, и выполнил все задания. Запросы, которые ты использовал для получения необходимых данных из базы, были составлены оптимально. 
    
---
 
Есть несколько критичных моментов, которые необходимо доработать, чтобы проект был принят. Они описаны в ячейках с красным фоном по ходу тетрадки и указаны ниже:
    
- удалить дублирующиеся строки с созданием подключения
- доработать запрос на определение автора с самой высокой средней оценкой книг

    
Кроме того, рекомендую доработать желтые комментарии, но оставляю это на твое усмотрение. 
    
Желаю успехов и буду ждать проект на повторную проверку.

# Проект по SQL

### Цель исследования
Первая задача как аналитика — проанализировать базу данных.
Эти данные помогут сформулировать ценностное предложение для нового продукта.

### Общие данные:
База данных содержит информацию о книгах, издательствах, авторах, а также пользовательские
обзоры книг. 

### Описание таблиц:
##### books cодержит данные о книгах:   
 - book_id — идентификатор книги;   
 - author_id — идентификатор автора;   
 - title — название книги;   
 - num_pages — количество страниц;   
 - publication_date — дата публикации книги;   
 - publisher_id — идентификатор издателя.   

##### authors cодержит данные об авторах:   
 - author_id — идентификатор автора;   
 - author — имя автора    

##### publishers cодержит данные об издательствах:    
 - publisher_id — идентификатор издательства;   
 - publisher — название издательства;   

##### ratings cодержит данные о пользовательских оценках книг:   
 - rating_id — идентификатор оценки;   
 - book_id — идентификатор книги;   
 - username — имя пользователя, оставившего оценку;   
 - rating — оценка книги.   

##### reviews cодержит данные о пользовательских обзорах на книги:   
 - review_id — идентификатор обзора;   
 - book_id — идентификатор книги;   
 - username — имя пользователя, написавшего обзор;   
 - text — текст обзора.   

<div class="alert alert-success"; style="border-left: 7px solid green">
<b>✅ Комментарий ревьюера, ревью 1</b> 

Отлично, что начинаешь со введения.

### Подключение к БД

In [1]:
# импортируем библиотеки
import pandas as pd
from sqlalchemy import text, create_engine
import warnings; warnings.filterwarnings(action='once')

In [2]:
# устанавливаем параметры
db_config = {'user': 'praktikum_student', # имя пользователя
            'pwd': 'Sdf4$2;d-d30pp', # пароль
            'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
            'port': 6432, # порт подключения
            'db': 'data-analyst-final-project-db'} # название базы данных
connection_string = 'postgresql://{user}:{pwd}@{host}:{port}/{db}'.format(**db_config)
# сохраняем коннектор
engine = create_engine(connection_string, connect_args={'sslmode':'require'})
con=engine.connect()

<div class="alert alert-success"; style="border-left: 7px solid green">
<b>✅ Комментарий ревьюера, ревью 2</b> 


    
✔️ 
    
<a class="tocSkip"></a><s>

<div class="alert alert-warning"; style="border-left: 7px solid gold">
<b>⚠️ Комментарий ревьюера, ревью 1</b> 

У тебя дублируется импорт библиотек. Думаю, ты просто не заметил. На всякий случай, напомню, что стоит импортировать все библиотеки в отдельной - от загрузки датасетов и других действий - ячейке. В таком случае при необходимости импортировать новую библиотеку в процессе работы над проектом тебе не придется заново загружать датасеты / выполнять другие операции, указанные в ячейке.
    
<a href='https://pythonworld.ru/osnovy/pep-8-rukovodstvo-po-napisaniyu-koda-na-python.html'>Материал по импорту.</a> 

### Знакомство с таблицами

#### Таблица books

In [3]:
# чтобы выполнить SQL-запрос, используем Pandas
query = '''SELECT * FROM books LIMIT 5'''
pd.io.sql.read_sql(sql=text(query), con = con)

,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268


<div class="alert alert-success"; style="border-left: 7px solid green">
<b>✅ Комментарий ревьюера, ревью 2</b> 


    
✔️ 
    
<a class="tocSkip"></a><s>

<div class="alert alert-danger"; style="border-left: 7px solid red">
<b>⛔️ Комментарий ревьюера, ревью 1</b> 

Нет необходимости каждый раз заново создавать подключение к базе. Достаточно это сделать один раз. А далее можно использовать переменную с сохраненным коннектором.

<div class="alert alert-info"; style="border-left: 7px solid blue">
<b> Да верно, торопился не обратил внимание. Поправил.</b>

#### Таблица authors

In [4]:
# чтобы выполнить SQL-запрос, используем Pandas
query = '''SELECT * FROM authors LIMIT 5'''
pd.io.sql.read_sql(sql=text(query), con = con)

,author_id,author
0,1,A.S. Byatt
1,2,Aesop/Laura Harris/Laura Gibbs
2,3,Agatha Christie
3,4,Alan Brennert
4,5,Alan Moore/David Lloyd


#### Таблица publishers

In [5]:
# чтобы выполнить SQL-запрос, используем Pandas
query = '''SELECT * FROM publishers LIMIT 5'''
pd.io.sql.read_sql(sql=text(query), con = con)

,publisher_id,publisher
0,1,Ace
1,2,Ace Book
2,3,Ace Books
3,4,Ace Hardcover
4,5,Addison Wesley Publishing Company


#### Таблица ratings

In [6]:
# чтобы выполнить SQL-запрос, используем Pandas
query = '''SELECT * FROM ratings LIMIT 5'''
pd.io.sql.read_sql(sql=text(query), con = con)

,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5
3,4,2,lorichen,3
4,5,2,mariokeller,2


#### Таблица reviews

In [7]:
# чтобы выполнить SQL-запрос, используем Pandas
query = '''SELECT * FROM reviews LIMIT 5'''
pd.io.sql.read_sql(sql=text(query), con = con)

,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Amo...
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but ...
3,4,3,johnsonamanda,Finally month interesting blue could nature cu...
4,5,3,scotttamara,Nation purpose heavy give wait song will. List...


<div class="alert alert-success"; style="border-left: 7px solid green">
<b>✅ Комментарий ревьюера, ревью 1</b> 

Отлично, таблицы изучены.

### Задачи

#### 1. Посчитайте, сколько книг вышло после 1 января 2000 года :

In [8]:
query = '''
        SELECT COUNT(book_id) AS КнигСНачала2000года
        FROM books
        WHERE publication_date > '2000-01-01'
'''
pd.io.sql.read_sql(query, con = con)

,КнигСНачала2000года
0,819


<div class="alert alert-success"; style="border-left: 7px solid green">
<b>✅ Комментарий ревьюера, ревью 2</b> 


    
✔️ ОК, поняла. Вариант с text специально дается в рекомендациях к уроку, чтобы работало у ревьюеров при проверке в разных инструментах (например, в Google Colab).
    
Но менять нет необходимости, у меня работает и в таком виде.
    
<a class="tocSkip"></a><s>

<div class="alert alert-warning"; style="border-left: 7px solid gold">
<b>⚠️ Комментарий ревьюера, ревью 1</b> 



Обратила внимание, что ты используешь разный синтаксис:
    
- в предыдущих запросах `pd.io.sql.read_sql(sql=text(query), con = con)`
    
- с текущего и далее `pd.io.sql.read_sql(query, con = engine)`
    
Можешь объяснить, почему?

<div class="alert alert-info"; style="border-left: 7px solid blue">
<b> Запрос с sql=text, был в шаблоне подключения в описании к проекту ctrl+c\ctrl+v. Далее делал запросы в Visual code помещая в query сам менял\удалял\добавлял, по итогу оставил так (больше кода писать). Обязательно оставлять с sql=text?</b>

Начиная с 1 января 2000 года в каталог сервиса было добавлено 819 книг.

<div class="alert alert-success"; style="border-left: 7px solid green">
<b>✅ Комментарий ревьюера, ревью 1</b> 

Количество книг посчитано верно.

#### 2. Для каждой книги посчитайте количество обзоров и среднюю оценку :

In [9]:
query = '''
       SELECT b.book_id, 
              b.title AS Заголовок, 
              COUNT(DISTINCT rv.review_id) AS Обзоров, 
              ROUND(AVG(r.rating),3) AS СреднийРейтинг
       FROM books AS b
       LEFT JOIN reviews AS rv ON b.book_id = rv.book_id
       LEFT JOIN  ratings AS r ON b.book_id = r.book_id
       GROUP BY b.book_id, b.title
       ORDER BY Обзоров DESC
'''

pd.io.sql.read_sql(query, con = con)

,book_id,Заголовок,Обзоров,СреднийРейтинг
0,948,Twilight (Twilight #1),7,3.663
1,963,Water for Elephants,6,3.977
2,734,The Glass Castle,6,4.207
3,302,Harry Potter and the Prisoner of Azkaban (Harr...,6,4.415
4,695,The Curious Incident of the Dog in the Night-Time,6,4.081
...,...,...,...,...
995,83,Anne Rice's The Vampire Lestat: A Graphic Novel,0,3.667
996,808,The Natural Way to Draw,0,3.000
997,672,The Cat in the Hat and Other Dr. Seuss Favorites,0,5.000
998,221,Essential Tales and Poems,0,4.000


По данным можно сказать что : в каталоге 1000 книг. Минимальное количество обзоров от 0 до 7. Оценки рейтинга судя по данным не слишком зависят от количества обзоров (по количеству не сказать о качестве этих обзоров). Средний рейтиинг всех книг примерно 4.

<div class="alert alert-success"; style="border-left: 7px solid green">
<b>✅ Комментарий ревьюера, ревью 1</b> 

Ты хорошо построил запрос - сохранил все книги из таблицы books (даже те, для которых нет обзоров), но при этом учел только уникальные обзоры.

#### 3. Определите издательство, которое выпустило наибольшее число книг толще 50 страниц — так вы исключите из анализа брошюры;

In [10]:
query = '''
        WITH w AS (
        SELECT publisher_id, book_id
        FROM books
        WHERE num_pages > 50
        )
        SELECT p.publisher AS Издательство
        FROM w 
        INNER JOIN publishers AS p ON w.publisher_id = p.publisher_id
        GROUP BY p.publisher
        ORDER BY COUNT(w.book_id) DESC
        LIMIT 1
'''
pd.io.sql.read_sql(query, con = con)

,Издательство
0,Penguin Books


Большее количество представленных книг с тольшиной более чем 50 страниц находятся у издательства Penguin Books.

<div class="alert alert-success"; style="border-left: 7px solid green">
<b>✅ Комментарий ревьюера, ревью 1</b> 


    
Издательство определено верно.
    
<a class="tocSkip"></a><s>



#### 4. Определите автора с самой высокой средней оценкой книг — учитывайте только книги с 50 и более оценками;

In [17]:
query = '''
       WITH w AS (
       SELECT book_id
       FROM ratings
       GROUP BY book_id
       HAVING COUNT(rating_id) >= 50
       )
       SELECT a.author AS Автор,
              AVG(r.rating) AS СреднийРейтинг
       FROM books AS b
       INNER JOIN w ON b.book_id = w.book_id
       INNER JOIN authors AS a ON b.author_id = a.author_id
       INNER JOIN ratings AS r ON w.book_id = r.book_id
       GROUP BY a.author_id, a.author
       ORDER BY AVG(r.rating) DESC
       LIMIT 1
'''
pd.io.sql.read_sql(query, con = con)

,Автор,СреднийРейтинг
0,J.K. Rowling/Mary GrandPré,4.287097


Самые популярный авторы с средним рейтингом 4.29 оказались Джоан Роулинг и Мари Грандпре. Книга Гарри поттер и узник Азкабана.

<div class="alert alert-warning"; style="border-left: 7px solid gold">
<b>⚠️ Комментарий ревьюера, ревью 3</b> 

Запрос составлен верно. Только в выводе требуется корректировка - мы определяем автора с максимальным рейтингом, а не книгу. 

    
<a class="tocSkip"></a><s>

<div class="alert alert-danger"; style="border-left: 7px solid red">
<b>⛔️ Комментарий ревьюера, ревью 2</b> 
    
Не нужно было переписывать запрос, он был составлен верно: нам необходимо определить топ-автора по средним оценкам книг, а не топ-книгу как у тебя сейчас. 
    
Я имела в виду, что по условиям задачи в CTE должно быть не > 50, а >= 50. Можешь, пожалуйста, доработать запрос?

<div class="alert alert-danger"; style="border-left: 7px solid red">
<b>⛔️ Комментарий ревьюера, ревью 1</b> 

Хоть это и не повлияет на результат, но стоит все-таки привести запрос в соответствие с условием. По условию книги с 50 оценками должны тоже участвовать в анализе. Пожалуйста, доработай запрос.

#### 5. Посчитайте среднее количество обзоров от пользователей, которые поставили больше 48 оценок.

In [12]:
query = '''
        SELECT COUNT(DISTINCT rv.text)/ COUNT(DISTINCT rv.username) AS avg_rv_count
        FROM ratings AS r
        INNER JOIN reviews AS rv ON r.username = rv.username
        WHERE r.username in (SELECT username
                            FROM ratings
                            GROUP BY username
                            HAVING COUNT(rating_id) > 48)
'''
pd.io.sql.read_sql(query, con = con)

,avg_rv_count
0,24


Среднее число обзоров является 24, среди тех пользователей которые поставивили в среднем болеее 48 оценок.

<div class="alert alert-success"; style="border-left: 7px solid green">
<b>✅ Комментарий ревьюера, ревью 1</b> 

Все верно.

<div class="alert alert-success"; style="border-left: 7px solid green">
<b>✅ Комментарий ревьюера, ревью 2</b> 


    
✔️ 
    
<a class="tocSkip"></a><s>

<div class="alert alert-warning"; style="border-left: 7px solid gold">
<b>⚠️ Комментарий ревьюера, ревью 1</b> 

В конце каждого проекта стоит оставлять общий вывод, кратко резюмируя выполненные задачи и полученные результаты.

Общий вывод: Магазин книг должен сосредоточиться на расширении ассортимента, например книг издательства Penguin Books, и активно продвигать популярные книги, а так же авторов похожих на Джоан Роулинг и Мари Грандпре. Также следует поощрять и привлекать пользователей, которые активно оценивают книги.